In [1]:
from google.colab import drive
import pandas as pd

# Mount Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
path = '/content/drive/My Drive/preprocessed_data.csv'

In [3]:
preprocessed_data = pd.read_csv(path)

In [4]:
preprocessed_data.head()

,comment_text,Score
0,explanation edits made username hardcore metal...,0
1,aww match background colour seemingly stuck th...,0
2,hey man really trying edit war guy constantly ...,0
3,make real suggestion improvement wondered sect...,0
4,sir hero chance remember page,0


In [5]:
!pip install transformers 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 51.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 113.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 26.6 MB/s eta 0:00:00


In [6]:
from transformers import RobertaTokenizerFast
from transformers import TFRobertaForSequenceClassification
import numpy as np

import tensorflow as tf

In [7]:
preprocessed_data.dropna(inplace=True)

In [8]:
preprocessed_data.dropna(subset=['comment_text'], inplace=True)

In [9]:
preprocessed_data.dropna(subset=['Score'], inplace=True)

In [10]:
comments = preprocessed_data['comment_text'].values.tolist()
labels = preprocessed_data['Score'].values.tolist()


In [11]:
from sklearn.model_selection import train_test_split
training_sentences , validation_sentences , training_labels , validation_labels = train_test_split(comments,labels,test_size=0.2)

In [12]:
tokenizer = RobertaTokenizerFast.from_pretrained('roberta-base')

tokenizer([training_sentences[0]],truncation=True,padding=True,max_length=128)

{'input_ids': [[0, 12338, 741, 3961, 1851, 9740, 579, 30453, 337, 464, 156, 94, 1946, 14156, 90, 352, 1593, 364, 821, 7636, 12764, 1037, 1035, 1021, 31434, 2857, 2446, 6456, 1593, 464, 156, 1842, 1236, 922, 16080, 438, 2]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}

In [13]:
train_encodings=tokenizer(training_sentences,truncation=True,padding=True)
val_encodings=tokenizer(validation_sentences,truncation=True,padding=True)


In [14]:
train_dataset = tf.data.Dataset.from_tensor_slices((dict(train_encodings), training_labels))
val_dataset = tf.data.Dataset.from_tensor_slices((dict(val_encodings), validation_labels))

In [15]:
model=TFRobertaForSequenceClassification.from_pretrained('roberta-base',num_labels=2)

All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

Some layers of TFRobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [16]:
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5,epsilon=1e-8)
model.compile(optimizer=optimizer, loss=model.compute_loss, metrics=['accuracy'])

In [18]:
test_sentence = 'die hate prick dint deserve place'

In [19]:
predict_input=tokenizer.encode(test_sentence,truncation=True,padding=True,return_tensors='tf')

In [20]:
tf_output=model.predict(predict_input)[0]
tf_prediction= tf.nn.softmax(tf_output,axis=1)
labels=['Okay','Alert']
label=tf.argmax(tf_prediction,axis=1)
label=label.numpy()
print(labels[label[0]])

1/1 [==============================] - 3s 3s/step
Alert


In [17]:
model.fit(train_dataset.shuffle(100).batch(16),
          epochs=2,
          batch_size=16,
          validation_data=val_dataset.shuffle(100).batch(16))

Epoch 1/2


AttributeError: ignored